# In Dimension Reduction Part2


## Compare Mutiple Reduction Methods 
    - pca, kernel pca, isomap, lle
    
====================================================================================================================

### Importing Modules

In [1]:
import pandas as pd
from sklearn.decomposition import PCA, KernelPCA
from sklearn import manifold
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

### Import Datasets

In [2]:
path='ARCENE\\'

#import datasets
frame='arcene_train.data' 
D_train= pd.read_table(path+frame,sep='\s',names=['X'+str(i) for i in range(1,10001)])

frame='arcene_valid.data'
D_valid= pd.read_table(path+frame,sep='\s',names=['X'+str(i) for i in range(1,10001)])

frame='arcene_train.labels.txt'
D_L_train= pd.read_table(path+frame,sep='\t',names=['Target'])

frame='arcene_valid.labels.txt'
D_L_valid= pd.read_table(path+frame,sep='\t',names=['Target'])

X=D_train.append(D_valid)

C:\Users\jeonyunsu\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
C:\Users\jeonyunsu\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


### Implement KNN Classifier With No Dimension Reduction Method

In [3]:
knn = BaggingClassifier(KNeighborsClassifier(n_neighbors=5))
knn.fit(D_train,D_L_train.values.ravel())
pred_knn=knn.predict(D_valid)
ACC_origin=round((sum(np.where(pred_knn==D_L_valid['Target'],1,0))/len(D_L_valid))*100,2) # Basic Knn Accuracy

print(ACC_origin)

83.0


### Implement KNN Classifier With Mutiple Dimension Reduction Methods

Explanation 

    #========================================적절한 Parameter 찾기==================================================#
    #Kernel PCA의 경우 적용시킬 다양한 함수와 감마값에 따라 성능차이가 크게 났다. 데이터 분포 알맞는 커널을 찾으면 성능이 더 잘 나올 수도 있을 것.
    #이번에는 Poly, RBF, Cosine, Sigmoid를 비교해본 결과 cosine에서 성능이 괜찮게나와서 cosine으로 선택하게 되었다.

    #Isomap이나 LLE 같은 경우에는 적절한 K를 찾는 것이 성능을 더 높일 수 있는 관건일 것이다. 5,10,15으로 K값을 조절해가며 돌린결과 
    #K=5가 평균적으로 높은 정확도를 보이는 경향이 있어 선택하게 되었다.  

    #========================================KernelPCA=================================================#
    #pred=[]
    #gammas=[1,10,50,80,100]
    #for n_components in dim:
    #    for g in gammas:
    #        #Kernel PCA(rbf)
    #        kpca=KernelPCA(n_components=n_components,kernel='rbf',gamma=g)
    #        x_kpca=kpca.fit_transform(X)
    #        knn.fit(x_kpca[0:100,:],D_L_train)
    #        pred_knn_KP1=knn.predict(x_kpca[100:200,:])
    #        
    #        #Kernel PCA(poly)        
    #        kpca=KernelPCA(n_components=n_components,kernel='poly',gamma=g)
    #        x_kpca=kpca.fit_transform(X)
    #        knn.fit(x_kpca[0:100,:],D_L_train)
    #        pred_knn_KP2=knn.predict(x_kpca[100:200,:])
    #
    #        pred_knns=[pred_knn_KP1,pred_knn_KP2]
    #        A=[round((sum(np.where(pred_knn==D_L_valid['Target'],1,0))/len(D_L_valid))*100,2) for pred_knn in pred_knns]
    #        pred.append(A)

    #np.argmax(pred,axis=0) #최댓값은 dim 50일때와 gamma가 10일때, ploy의 정확도 81%

    #pred2=[]
    #for n_components in dim:
    #    #Kernel PCA(cosine)
    #    kpca=KernelPCA(n_components=n_components,kernel='cosine')
    #    x_kpca=kpca.fit_transform(X)
    #    knn.fit(x_kpca[0:100,:],D_L_train)
    #    pred_knn_KP3=knn.predict(x_kpca[100:200,:])
    #    
    #    #Kernel PCA(sigmoid)
    #    kpca=KernelPCA(n_components=n_components,kernel='sigmoid')
    #    x_kpca=kpca.fit_transform(X)
    #    knn.fit(x_kpca[0:100,:],D_L_train)
    #    pred_knn_KP4=knn.predict(x_kpca[100:200,:])
    #    
    #    pred_knns=[pred_knn_KP3,pred_knn_KP4]
    #    A=[round((sum(np.where(pred_knn==D_L_valid['Target'],1,0))/len(D_L_valid))*100,2) for pred_knn in pred_knns]
    #    pred2.append(A)

    #np.argmax(pred2,axis=0)#최댓값은 dim 50일때, cosine 정확도 82%
    #따라서 Cosine 선택
    #========================================Isomap, LLE=================================================#
    #ACC_set=[]
    #dim=[10,50,100]
    #n_nes=[5,10,15]
    #for n_components in dim:
    #    for n_neighbors in n_nes:
    #       #Isomap
    #        isomap=manifold.Isomap(n_neighbors, n_components)
    #        x_isomap=isomap.fit_transform(X)
    #        knn.fit(x_isomap[0:100,:],D_L_train)
    #        pred_knn_I=knn.predict(x_isomap[100:200,:])
    #        
    #        #lle
    #        lle=manifold.LocallyLinearEmbedding(n_neighbors, n_components)
    #        x_lle=lle.fit_transform(X)
    #        knn.fit(x_lle[0:100,:],D_L_train)
    #        pred_knn_L=knn.predict(x_lle[100:200,:])
    #        
    #        #calculate ACC
    #        reduct_methods=[pred_knn_I, pred_knn_L]
    #        ACC=[round((sum(np.where(pred_knn==D_L_valid['Target'],1,0))/len(D_L_valid))*100,2) for pred_knn in reduct_methods]
    #        ACC_set.append(ACC)
    # Isomap의 경우 모든차원에서 K=5일때 성능이 가장 좋았고, LLE의 경우 dim=50을 제외한 10,100차원에서 K=5일때 성능이 가장 좋았다. 
    # 따라서 K=5를 선택


- Implementing KNN Classifier

In [4]:
ACC_set=[]
dim=[10,50,100]
n_neighbors=5

for n_components in dim:
    
    #PCA
    pca=PCA(n_components)
    x_pca=pca.fit_transform(X)
    knn.fit(x_pca[0:100,:],D_L_train.values.ravel())
    pred_knn_P=knn.predict(x_pca[100:200,:])
    
    #Kernel PCA
    kpca=KernelPCA(n_components, kernel='cosine')
    x_kpca=kpca.fit_transform(X)
    knn.fit(x_kpca[0:100,:], D_L_train.values.ravel())
    pred_knn_KP=knn.predict(x_kpca[100:200,:])
    
    #Isomap
    isomap=manifold.Isomap(n_neighbors, n_components)
    x_isomap=isomap.fit_transform(X)
    knn.fit(x_isomap[0:100,:],D_L_train.values.ravel())
    pred_knn_I=knn.predict(x_isomap[100:200,:])
    
    #lle
    lle=manifold.LocallyLinearEmbedding(n_neighbors, n_components)
    x_lle=lle.fit_transform(X)
    knn.fit(x_lle[0:100,:],D_L_train.values.ravel())
    pred_knn_L=knn.predict(x_lle[100:200,:])
    
    #calculate ACC
    reduct_methods=[pred_knn_P, pred_knn_KP, pred_knn_I, pred_knn_L]
    ACC=[round((sum(np.where(pred_knn==D_L_valid['Target'],1,0))/len(D_L_valid))*100,2) for pred_knn in reduct_methods]
    ACC_set.append(ACC)

### Compare Performance

In [5]:
ACC_set=pd.DataFrame(np.array(ACC_set))
ACC_set.columns=['PCA','Kernel_PCA','Isomap','LLE']
ACC_set.index=dim

print(ACC_set[ACC_set>=ACC_origin]) #KNN의 기본보다 나은 성능을 보이는 차원감소 기법들 

      PCA  Kernel_PCA  Isomap  LLE
10    NaN         NaN     NaN  NaN
50    NaN         NaN    86.0  NaN
100  84.0         NaN    86.0  NaN
